Les imports

In [24]:
import sys

from tensorflow.keras.layers import Convolution2D  # traite les image ( vu qu'il sont en 2d)
from tensorflow.keras.layers import MaxPooling2D  # pour la récupération d'information importante
from tensorflow.keras.layers import Flatten  # feature map pour les applatir
from tensorflow.keras.layers import Dense  # ajout de couche
from tensorflow.keras.layers import Dropout  # probabilité pour "endormir" les neurone
from tensorflow.keras.models import Sequential  # initialise le réseau de neuronne

from keras.preprocessing.image import ImageDataGenerator

import math
import os

from commun.informationLearningWriter import *

Paramètres Généraux

In [17]:
dir = "CNN"  # le directory des photos

path2write = "./"


Paramètress du modèle de l'IA

In [18]:
nbrHiddenLayer = 3
neuroneCoucheCacher = 128
tailleLot = 32
nbrPoolConv = 4
fctActivation = 'relu'
optimizer = 'adam'
imageSize = 150
rateDropOut = 0.3
shearRange = 0.2
zoomRange = 0.2
activationConvolution = 'relu'
nbEpoque = 15
nbFiltre = 32
kernelSize = 3
stridesConv = 1
stridesPool = 2
allMetrics = ["acc"]
sizeMatricePool = (2, 2)
folderTrain = dir + "/" + "training_set"
folderTest = dir + "/" + "test_set"

outUnit = 1
fctLoss = "binary_crossentropy"
classMode = 'binary'

Comptage du nombre d'images

In [19]:
def comptageTot(folderPathTrain, folderPathTest):
    """ compte le nombre total de photo """
    imageTest = 0
    imageTrain = 0
    foldersTest = os.listdir(folderPathTest)
    foldersTrain = os.listdir(folderPathTrain)
    for subFolders in foldersTrain:
        imageTrain += comptageImage(folderPathTrain + "/" + subFolders)
    for subFolders in foldersTest:
        imageTest += comptageImage(folderPathTest + "/" + subFolders)
    return (imageTrain, imageTest)


def comptageImage(pathFolder):
    folder = os.listdir(pathFolder)
    return len(folder)


nbImageTrain, nbImageTest = comptageTot(folderTrain, folderTest)

Construction de l'IA

In [20]:
def buildModel():
    model = Sequential()

    # phase traitement image
    model.add(Convolution2D(filters=nbFiltre, kernel_size=kernelSize, strides=stridesConv, input_shape=(imageSize, imageSize, 3),
                            activation=activationConvolution))
    # Phase de convolution , on transforme l'image avec la fonction de convolution grace a des filtre pour avoir un set de featurs map
    model.add(MaxPooling2D(pool_size=sizeMatricePool, strides=stridesPool))
    # Phase de Pooling ( avec le max pooling ) , permet de réduire la taille en gardant l'information la plus importante ( pour les différente prise de vue)

    nbrLayer = 1
    multiplicateur = 1
    for i in range(nbrPoolConv - 1):
        model.add(Convolution2D(filters=nbFiltre * multiplicateur, kernel_size=kernelSize, strides=stridesConv,
                                activation=activationConvolution))
        model.add(MaxPooling2D(pool_size=sizeMatricePool, strides=stridesPool))
        nbrLayer += 1
        multiplicateur = math.floor((nbrLayer / 2) + 1)

    # phase feed forward
    model.add(Flatten())  # Couche d'entrée
    # Etape de flattening ( linéarise la matrice de pooling feature map pour le réseau de ANN)

    for j in range(nbrHiddenLayer):
        model.add(Dense(units=neuroneCoucheCacher, activation=fctActivation))

    ##Couche de sortie
    model.add(Dropout(rateDropOut))
    model.add(Dense(units=outUnit, activation="sigmoid"))

    model.compile(optimizer=optimizer, loss=fctLoss, metrics=allMetrics)

    return model



CNN = buildModel()

Generateur d'image (sert au rescaling)

In [22]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train = train_datagen.flow_from_directory(folderTrain,
                                          target_size=(imageSize,imageSize),
                                          batch_size= )

CNN/training_set


Entrainement de l'IA

In [21]:
saves = CNN.fit(
    folderTrain,
    steps_per_epoch=math.ceil(nbImageTrain / tailleLot),
    epochs=nbEpoque,
    validation_data=folderTest,
    validation_steps=math.ceil(nbImageTest / tailleLot),
    verbose=1)

IndexError: list index out of range

Sauvegarde des Information lier a l'apprentissage

In [ ]:
writeIntoFile(saves.history['val_acc'],saves.history['acc'],path2write)
